<a href="https://colab.research.google.com/github/chanukaDissanayaka/RideFareClassification/blob/master/Ridefare_submission_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import RobustScaler

from datetime import datetime, timedelta
from pandas import DataFrame

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

import numpy as np

RANDOM_SEED = 4

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#dataframe = pd.read_csv("/content/train.csv")
#test = pd.read_csv("/content/test.csv")

dataframe = pd.read_csv("/content/gdrive/My Drive/ML_RideFare/Data/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/ML_RideFare/Data/test.csv")

dataframe.isna().sum(axis = 0)
#dataframe = dataframe.fillna(method='ffill')
dataframe = dataframe.dropna()

In [ ]:
def preprocess_train_data(dataframe):
  feature_set = dataframe.iloc[:,1:13]

  features = feature_set

  features['pickup_time'] = pd.to_datetime(features['pickup_time'], format="%m/%d/%Y %H:%M")
  features['drop_time'] = pd.to_datetime(features['drop_time'], format="%m/%d/%Y %H:%M")

  td = features

  td['pickup_time_year'] = td['pickup_time'].dt.year
  td['pickup_time_month'] = td['pickup_time'].dt.month
  td['pickup_time_day'] = td['pickup_time'].dt.day
  td['pickup_time_hour'] = td['pickup_time'].dt.hour
  td['pickup_time_minute'] = td['pickup_time'].dt.minute

  td['drop_time_year'] = td['drop_time'].dt.year
  td['drop_time_month'] = td['drop_time'].dt.month
  td['drop_time_day'] = td['drop_time'].dt.day
  td['drop_time_hour'] = td['drop_time'].dt.hour
  td['drop_time_minute'] = td['drop_time'].dt.minute

  td['day_of_week'] = td['pickup_time'].dt.day_name()
  td['day_of_week'] = OrdinalEncoder().fit_transform(td[['day_of_week']])

  td['duration_calculated'] = td['drop_time'] - td['pickup_time']
  td['duration_calculated'] = td['duration_calculated'].dt.total_seconds().astype(int)

  td = td.drop('pickup_time',1)
  td = td.drop('drop_time',1)

  td = get_trip_distance_eucld(td)
  td = get_X_distance(td)
  td = get_Y_distance(td)
  td = get_pick_XY(td)
  td = get_drop_XY(td)
  td = transform_hour(td)
  td = set_fare_duration(td)
  td = set_driving_duration(td)
  td = get_exact_fare(td)
  td = combine_time_fare(td)
  

  return td

def get_labels(dataframe):
  label_set = dataframe.iloc[:, 13:14]
  #rev_label_set = label_set
  rev_label_set = label_set.replace({'correct':'incorrect', 'incorrect':'correct'})

  encoder = LabelEncoder()
  encoder.fit(rev_label_set)
  labels = encoder.transform(rev_label_set)
  return labels

def set_time_of_the_day(cleaned_train_set):
  cleaned_train_set.head()
  newFrame = cleaned_train_set
  tt = newFrame

  d0_index = newFrame[(newFrame['pickup_time_hour']>=0) & (newFrame['pickup_time_hour'] < 6 )].index
  tt.loc[d0_index,'time_of_day']=0

  d1_index = newFrame[(newFrame['pickup_time_hour']>=6) & (newFrame['pickup_time_hour'] < 12 )].index
  tt.loc[d1_index,'time_of_day']=1

  d2_index = newFrame[(newFrame['pickup_time_hour']>=12) & (newFrame['pickup_time_hour'] < 18 )].index
  tt.loc[d2_index,'time_of_day']=2

  d3_index = newFrame[(newFrame['pickup_time_hour']>=18) & (newFrame['pickup_time_hour'] < 24 )].index
  tt.loc[d3_index,'time_of_day']=3

  return tt

def haversine_distance(lat1, lon1, lat2, lon2):
  r = 6371
  phi1 = np.radians(lat1)
  phi2 = np.radians(lat2)
  d_phi = np.radians(lat2-lat1)
  d_lambda = np.radians(lon2-lon1)

  a = np.sin(d_phi /2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(d_lambda/2)**2
  res = r*(2* np.arctan2(np.sqrt(a), np.sqrt(1-a)))

  return np.round(res,2)
  return res

def get_trip_distance(frame):
  newFrame = frame
  newFrame["distance"] = haversine_distance(frame['pick_lat'],frame['drop_lat'],frame['pick_lon'],frame['drop_lon'])
  return newFrame

def get_trip_distance_eucld(frame):
  newFrame = frame
  newFrame["distance"] = (((frame['pick_lat']-frame['drop_lat'])**2)+((frame['pick_lon']-frame['drop_lon'])**2)**0.5)
  return newFrame

def get_X_distance(frame):
  newFrame = frame
  newFrame["distance_X"] = ((frame['pick_lat']-frame['drop_lat']))
  return newFrame

def get_Y_distance(frame):
  newFrame = frame
  newFrame["distance_Y"] = ((frame['pick_lon']-frame['drop_lon']))
  return newFrame

def get_pick_XY(frame):
  newFrame = frame
  newFrame["pick_X"] = np.cos(frame['pick_lat'])* np.cos(frame['pick_lon'])
  newFrame["pick_Y"] = np.cos(frame['pick_lat'])* np.sin(frame['pick_lon'])
  return newFrame

def get_drop_XY(frame):
  newFrame = frame
  newFrame["drop_X"] = np.cos(frame['drop_lat'])* np.cos(frame['drop_lon'])
  newFrame["drop_Y"] = np.cos(frame['drop_lat'])* np.sin(frame['drop_lon'])
  return newFrame

def transform_hour(frame):
  newFrame = frame
  newFrame["pick_hour_sin"] = np.sin(np.radians(3*frame['pickup_time_hour']))
  newFrame["drop_hour_sin"] = np.sin(np.radians(3*frame['drop_time_hour']))

  newFrame["pick_hour_cos"] = np.cos(np.radians(3*frame['pickup_time_hour']))
  newFrame["drop_hour_cos"] = np.cos(np.radians(3*frame['drop_time_hour']))
  return newFrame

def set_fare_duration(imputed_train_set):
  df = imputed_train_set

  df['total_fare'] = ((df['fare']-df['fare'].mean())/df['fare'].std() +
                      (df['additional_fare']-df['additional_fare'].mean())/df['additional_fare'].std()+ 
                      (df['meter_waiting_fare']-df['meter_waiting_fare'].mean())/df['meter_waiting_fare'].std())

  df['total_fare'] =  df['fare'] + df['additional_fare'] + df['meter_waiting_fare']
  df['total_additional_fare'] = df['additional_fare'] + df['meter_waiting_fare']
  df['total_waiting_time'] = df['meter_waiting'] + df['meter_waiting_till_pickup']
  df['total_time'] = df['total_waiting_time'] + df['duration']

  df['waiting_factor'] = ((df['meter_waiting']-df['meter_waiting'].mean())/df['meter_waiting'].std() +
                      (df['meter_waiting_fare']-df['meter_waiting_fare'].mean())/df['meter_waiting_fare'].std()+ 
                      (df['meter_waiting_till_pickup']-df['meter_waiting_till_pickup'].mean())/df['meter_waiting_till_pickup'].std())

  return df

def set_driving_duration(train_set):
  df = train_set

  df['driving_duration'] = df['duration'] - df['meter_waiting']
  return df

def get_exact_fare(train_set):
  df = train_set
  df['exact_fare'] = df['fare'] - df['additional_fare']- df['meter_waiting_fare']
  return df

def combine_time_fare(train_set):
  df = train_set

  df['additional'] = ((df['additional_fare']-df['additional_fare'].mean())/df['additional_fare'].std() +
                        (df['meter_waiting_till_pickup']-df['meter_waiting_till_pickup'].mean())/df['meter_waiting_till_pickup'].std())

  df['waiting'] = ((df['meter_waiting']-df['meter_waiting'].mean())/df['meter_waiting'].std() +
                        (df['meter_waiting_fare']-df['meter_waiting_fare'].mean())/df['meter_waiting_fare'].std())

  df['exact'] = ((df['exact_fare']-df['exact_fare'].mean())/df['exact_fare'].std() +
                        (df['driving_duration']-df['driving_duration'].mean())/df['driving_duration'].std())

  return df

In [ ]:
print(dataframe.label.value_counts())

correct      15442
incorrect     1526
Name: label, dtype: int64


In [ ]:
cleaned_train_set = preprocess_train_data(dataframe)
labels = get_labels(dataframe)
cleaned_train_set.head()
labels[:100]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [ ]:
from sklearn.impute import SimpleImputer

cleaned_train_set.duration.fillna(cleaned_train_set.duration_calculated, inplace=True)
cleaned_train_set.head()

imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
dframe = imp_mean.fit_transform(cleaned_train_set)
k = cleaned_train_set.keys()
imputed_train_set = cleaned_train_set
imputed_train_set[k] = dframe
imputed_train_set.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_year,pickup_time_month,pickup_time_day,pickup_time_hour,pickup_time_minute,drop_time_year,drop_time_month,drop_time_day,drop_time_hour,drop_time_minute,day_of_week,duration_calculated,distance,distance_X,distance_Y,pick_X,pick_Y,drop_X,drop_Y,pick_hour_sin,drop_hour_sin,pick_hour_cos,drop_hour_cos,total_fare,total_additional_fare,total_waiting_time,total_time,waiting_factor,driving_duration,exact_fare,additional,waiting,exact
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,2019.0,11.0,1.0,0.0,20.0,2019.0,11.0,1.0,0.0,34.0,0.0,840.0,0.022663,-0.04078,0.0210,-0.175519,-0.818213,-0.187363,-0.791950,0.000000,0.000000,1.000000,1.000000,280.8200,10.5000,120.0,954.0,-0.249539,778.0,259.8200,-0.142215,-0.139866,-0.172774
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,2019.0,11.0,1.0,0.0,56.0,2019.0,11.0,1.0,1.0,9.0,0.0,780.0,0.006875,-0.02784,0.0061,-0.173513,-0.805325,-0.174935,-0.788535,0.000000,0.052336,1.000000,0.998630,208.3500,10.5000,181.0,972.0,-0.092330,744.0,187.3500,0.016110,-0.140982,-0.436469
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,2019.0,11.0,1.0,1.0,8.0,2019.0,11.0,1.0,1.0,26.0,0.0,1080.0,0.050301,-0.02830,-0.0495,-0.197078,-0.786528,-0.154631,-0.778754,0.052336,0.052336,0.998630,0.998630,312.1400,10.5000,141.0,1228.0,-0.253348,1007.0,291.1400,-0.149000,-0.136890,0.048871
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,2019.0,11.0,1.0,2.0,27.0,2019.0,11.0,1.0,2.0,37.0,0.0,600.0,0.007603,-0.00178,-0.0076,-0.175582,-0.781100,-0.169414,-0.781369,0.104528,0.104528,0.994522,0.994522,108.4638,26.1638,339.0,937.0,-0.180187,327.0,56.1362,-0.133168,-0.079562,-1.092511
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,2019.0,11.0,1.0,5.0,38.0,2019.0,11.0,1.0,6.0,35.0,0.0,3420.0,0.078384,0.21537,0.0320,-0.139842,-0.644356,-0.195765,-0.780623,0.258819,0.309017,0.965926,0.951057,1075.5200,10.5000,294.0,3701.0,-0.125349,3225.0,1054.5200,-0.033649,-0.124242,3.761081


In [ ]:
imputed_train_set.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_year,pickup_time_month,pickup_time_day,pickup_time_hour,pickup_time_minute,drop_time_year,drop_time_month,drop_time_day,drop_time_hour,drop_time_minute,day_of_week,duration_calculated,distance,distance_X,distance_Y,pick_X,pick_Y,drop_X,drop_Y,pick_hour_sin,drop_hour_sin,pick_hour_cos,drop_hour_cos,total_fare,total_additional_fare,total_waiting_time,total_time,waiting_factor,driving_duration,exact_fare,additional,waiting,exact
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,2019.0,11.0,1.0,0.0,20.0,2019.0,11.0,1.0,0.0,34.0,0.0,840.0,0.022663,-0.04078,0.0210,-0.175519,-0.818213,-0.187363,-0.791950,0.000000,0.000000,1.000000,1.000000,280.8200,10.5000,120.0,954.0,-0.249539,778.0,259.8200,-0.142215,-0.139866,-0.172774
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,2019.0,11.0,1.0,0.0,56.0,2019.0,11.0,1.0,1.0,9.0,0.0,780.0,0.006875,-0.02784,0.0061,-0.173513,-0.805325,-0.174935,-0.788535,0.000000,0.052336,1.000000,0.998630,208.3500,10.5000,181.0,972.0,-0.092330,744.0,187.3500,0.016110,-0.140982,-0.436469
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,2019.0,11.0,1.0,1.0,8.0,2019.0,11.0,1.0,1.0,26.0,0.0,1080.0,0.050301,-0.02830,-0.0495,-0.197078,-0.786528,-0.154631,-0.778754,0.052336,0.052336,0.998630,0.998630,312.1400,10.5000,141.0,1228.0,-0.253348,1007.0,291.1400,-0.149000,-0.136890,0.048871
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,2019.0,11.0,1.0,2.0,27.0,2019.0,11.0,1.0,2.0,37.0,0.0,600.0,0.007603,-0.00178,-0.0076,-0.175582,-0.781100,-0.169414,-0.781369,0.104528,0.104528,0.994522,0.994522,108.4638,26.1638,339.0,937.0,-0.180187,327.0,56.1362,-0.133168,-0.079562,-1.092511
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,2019.0,11.0,1.0,5.0,38.0,2019.0,11.0,1.0,6.0,35.0,0.0,3420.0,0.078384,0.21537,0.0320,-0.139842,-0.644356,-0.195765,-0.780623,0.258819,0.309017,0.965926,0.951057,1075.5200,10.5000,294.0,3701.0,-0.125349,3225.0,1054.5200,-0.033649,-0.124242,3.761081


In [ ]:
numeric_cols = ['meter_waiting','meter_waiting_fare','meter_waiting_till_pickup',
               'additional_fare','duration','pick_lat','pick_lon','drop_lat','drop_lon',
               'fare','pickup_time_day','pickup_time_hour','pickup_time_minute']

numeric_cols = ['meter_waiting','meter_waiting_fare','meter_waiting_till_pickup',
               'additional_fare','driving_duration','exact_fare','distance']
categorical_cols = ['pickup_time_hour',]

used_cols = ['additional_fare', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup',
             'distance','distance_X', 'distance_Y','driving_duration','exact_fare','pickup_time_hour']



In [ ]:
X_train, X_eval, Y_train, Y_eval = train_test_split(
    imputed_train_set,
    labels,
    test_size=0.20,
    shuffle=True,
    stratify=labels,
    random_state=RANDOM_SEED
)



In [ ]:
numeric_preprocessing_steps = Pipeline([
    ('minmax_scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OrdinalEncoder())
    ])


preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols),
        ("categorical", categorical_transformer, categorical_cols)
    ],
    remainder = "drop"
)

data_pipeline = Pipeline([
    ("preprocessor", preprocessor)
])


In [ ]:
data_pipeline.fit(X_train)
d_train = data_pipeline.transform(X_train)
d_eval = data_pipeline.transform(X_eval)

In [ ]:
print(d_train.shape, d_eval.shape)
Y_train.shape

(13574, 8) (3394, 8)


(13574,)

In [ ]:
d_train[0]

array([-7.52351097e-02, -4.11694089e-03,  2.79661017e-01,  0.00000000e+00,
       -3.11783107e-01, -2.51272690e-01, -5.13883182e-01,  1.10000000e+01])

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight
sample_w = compute_sample_weight(class_weight={0:5, 1:1}, y=Y_train)

In [ ]:
sample_w[:100]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
       10.,  1.,  1.,  1.,  1.,  1.,  1., 10.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., 10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1., 10.,  1.,  1., 10.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.,  1.,
        1., 10.,  1.,  1., 10.,  1.,  1.,  1., 10.,  1.,  1.,  1.,  1.,
        1.,  1., 10.,  1.,  1.,  1.,  1.,  1.,  1.])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score

gbclf = GradientBoostingClassifier(random_state=2, n_estimators = 1000, max_depth =10, 
                                   subsample = 1.0, verbose = 1)
gbclf.fit(d_train, Y_train, )
print(gbclf.score(d_eval, Y_eval))
pr = gbclf.predict(d_eval)
print(f1_score(Y_eval, pr, average='macro'))
confusion_matrix(Y_eval, pr)

      Iter       Train Loss   Remaining Time 
         1           0.4917            1.50m
         2           0.4463            1.55m
         3           0.4150            1.53m
         4           0.3867            1.52m
         5           0.3636            1.51m
         6           0.3430            1.51m
         7           0.3278            1.51m
         8           0.3144            1.51m
         9           0.2998            1.51m
        10           0.2912            1.50m
        20           0.2065            1.50m
        30           0.1486            1.48m
        40           0.1082            1.47m
        50           0.0770            1.45m
        60           0.0532            1.43m
        70           0.0415            1.42m
        80           0.0340            1.40m
        90           0.0287            1.39m
       100           0.0237            1.37m
       200           0.0047            1.21m
       300           0.0009            1.05m
       40

array([[ 176,  129],
       [  24, 3065]])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbclf = GradientBoostingClassifier(random_state=0, n_estimators = 850, max_depth =8, subsample = 1.0, loss ='exponential')
#gbclf.fit(d_train, Y_train, sample_w)
gbclf.fit(d_train, Y_train,sample_w)
gbclf.score(d_eval, Y_eval)

0.9560989982321745

In [ ]:
data_pipeline.fit(imputed_train_set)
d_all = data_pipeline.transform(imputed_train_set)
y_all = labels
print(d_all.shape, y_all.shape)

from sklearn.utils.class_weight import compute_sample_weight
sample_w = compute_sample_weight(class_weight={0:1, 1:1}, y=y_all)

gbclf = GradientBoostingClassifier(random_state=2, n_estimators = 850, max_depth =10, 
                                   subsample = 1.0, verbose = 1)

gbclf.fit(d_all, y_all)

(16968, 8) (16968,)
      Iter       Train Loss   Remaining Time 
         1           0.4970            1.56m
         2           0.4515            1.57m
         3           0.4204            1.56m
         4           0.3919            1.57m
         5           0.3708            1.58m
         6           0.3532            1.59m
         7           0.3375            1.58m
         8           0.3238            1.58m
         9           0.3103            1.58m
        10           0.2995            1.58m
        20           0.2159            1.56m
        30           0.1623            1.55m
        40           0.1236            1.53m
        50           0.0932            1.51m
        60           0.0708            1.49m
        70           0.0502            1.47m
        80           0.0415            1.45m
        90           0.0359            1.43m
       100           0.0301            1.41m
       200           0.0078            1.21m
       300           0.0019       

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=850,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=1,
                           warm_start=False)

In [ ]:
cleaned_test_data = preprocess_train_data(test)

test_set = data_pipeline.transform(cleaned_test_data)

In [ ]:
test_set.shape

(8576, 8)

In [ ]:
preds = gbclf.predict(test_set)
print(preds)

[1 0 1 ... 1 1 1]


In [ ]:
preds.shape

(8576,)

In [ ]:
output = pd.DataFrame(test,columns=['tripid'])
output['prediction'] = preds
output.head()

,tripid,prediction
0,213284604,1
1,213286352,0
2,213293973,1
3,213294622,1
4,213298687,1


In [ ]:
output.to_csv('/content/Ridefare_submission_27.csv',index=False)